###Project 1: Analyzing German Trade Data with regard to Live Sea Fishs

The aim of my analysis is to research how the trade with live ornamental marine fishes has changed within the last years in Germany: How did the trade volume develop? From which countries are thoses fishes imported? With this, I would like to find out whether there are fewer wild cashes than in earlier years.

STEP 1: IMPORT ALL NECESSARY PACKAGES

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
import xmltodict
from io import StringIO
import numpy as np
from deep_translator import GoogleTranslator

STEP 2: SET UP AN API-ACCOUNT 

I am retrieving trade data from the German Statistical Office. For this, I'll need an API Key to authorize myself.

In [2]:
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'
TOKEN = os.getenv('api_key')

In [3]:
headers = {
'Content-Type': 'application/x-www-form-urlencoded',
'username': TOKEN,
'password': ""
}
langPref = "de"

In [4]:
hello = requests.post(BASE_URL + 'helloworld/logincheck',
headers = headers,
data = {
'language': langPref
})
hello.text[:-36]

'{"Status":"Sie wurden erfolgreich an- und abgemeldet! Bei mehr als 3 parallelen Requests wurden länger als 15 Minuten laufende Requests beendet.","Username"'

STEP 3: DOWNLOAD THE FIRST DATASET VIA THE API

Name of dataset: "Exports and imports (foreign trade): Germany, years, list of goods (8-digit" ("Aus- und Einfuhr (Außenhandel): Deutschland, Jahre, Warenverzeichnis (8-Steller)")

Internal code according to website: 51000-13.

As it is too big, I have to retrieve it in two steps. The first request gives me the export code for the second request ("Exportkennung").

In [ ]:
#only run once
#url = f"https://www-genesis.destatis.de/genesisWS/web/ExportService_2010?method=TabellenExport&kennung={TOKEN}&namen=51000-0013&bereich=Alle&format=csv&strukturinformation=false&komprimieren=false&transponieren=false&startjahr=2010&endjahr=2012&zeitscheiben=&regionalmerkmal=&regionalschluessel=&sachmerkmal=&sachschluessel=&sachmerkmal2=&sachschluessel2=&sachmerkmal3=&sachschluessel3=&stand=&auftrag=true&sprache=de"
#response = requests.get(url)
#response.text

In [5]:
url = f"https://www-genesis.destatis.de/genesisWS/web/ExportService_2010?method=ErgebnisExport&kennung={TOKEN}&name=51000-0013_109613960&bereich=Meine&format=csv&komprimieren=false&sprache=de"
response = requests.get(url)
json_data = xmltodict.parse(response.content)

STEP 4: CREATING A DATAFRAME OUT OF THE RESPONSE

The response gives us a JSON with embedded CSV-code. Therefore, I'm first filtering for the CSV code and then creating a dataframe out of it.

In [6]:
table_data = json_data['soapenv:Envelope']['soapenv:Body']['ErgebnisExportResponse']['ErgebnisExportReturn']['tabelle']['tabellenDaten']

In [7]:
df = pd.read_csv(StringIO(table_data), sep=";", skiprows=5)

STEP 4: TIDY THE DATASET UP

In [8]:
#Rename Variables
df.rename(columns={'Unnamed: 0': 'Code', 'Unnamed: 1': 'Product', 'Ausfuhr: Besondere Maßeinheit': 'Export (number)', 'Ausfuhr: Gewicht': 'Export: Weight (t)', 'Ausfuhr: Wert': 'Export Value (Tsd. EUR)', 'Ausfuhr: Wert (US-Dollar)':'Export Value (Tsd. USD)','Einfuhr: Besondere Maßeinheit': 'Import (number)', 'Einfuhr: Gewicht': 'Import Weight (t)','Einfuhr: Wert': 'Import Value (Tsd. EUR)', 'Einfuhr: Wert (US-Dollar)':'Import Value (Tsd. USD)' },inplace=True)

In [9]:
#Drop second row (which has no values)
df.drop(0, inplace=True)

In [10]:
#Drop last three rows (which have no values)
end_index = df.query("Code == '__________'").index[0]
df = df.iloc[:end_index]
df = df[df["Code"] != "__________"]

In [11]:
#Filter for products/years I need
rows = ['WA03011900','2011', '2012', '2013', '2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024' '2025']
df = df[df['Code'].isin(rows)]

In [12]:
#Create a separate variable "year"
df['Year'] = df['Code']
df['Year'] = df['Year'].str.replace(r"^WA.*$",'NaN',regex=True)
df['Year'].replace(['NaN'], np.nan, inplace=True)
df['Year'] = df['Year'].ffill()

/var/folders/z0/7y9h036n67l9188jq204ym500000gn/T/ipykernel_2638/1045686315.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Year'].replace(['NaN'], np.nan, inplace=True)


In [13]:
#Get rid of the rows with only years in them
rows = ['WA03011900']
df = df[df['Code'].isin(rows)]

In [14]:
#Replace Numbers in European format to US format
df['Import Weight (t)'] = df['Import Weight (t)'].replace(r'\,', '.', regex=True)
df['Export: Weight (t)'] = df['Export: Weight (t)'].replace(r'\,', '.', regex=True)

In [15]:
#Filter for the last 10 years (as these are the ones I am interested in)
years = ['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
df = df[df['Year'].isin(years)]

In [16]:
#Get rid of the last row to get data until 2023 (to be comparable with other datasets only reaching until 2023)
df.drop(186578, inplace=True)

In [17]:
#Keep only the variables I want to use for my analysis
df = df[['Year','Import Value (Tsd. USD)','Import Weight (t)']]

STEP 5: SAVE MY FIRST DATASET AS CSV

In [18]:
df.to_csv('fishes.csv', index=False)

STEP 6: DOWNLOAD THE SECOND DATASET VIA API

Name of dataset: "Exports and imports (foreign trade): Germany, years, country, list of goods (8-digit" ("Aus- und Einfuhr (Außenhandel): Deutschland, Jahre, Land. Warenverzeichnis (8-Steller)") 

Internal code according to website: 51000-15.

In [19]:
url = f"https://www-genesis.destatis.de/genesisWS/web/ExportService_2010?method=TabellenExport&kennung={TOKEN}&namen=51000-0015&bereich=Alle&format=csv&strukturinformation=false&komprimieren=false&transponieren=false&startjahr=2013&endjahr=2024&zeitscheiben=&regionalmerkmal=STLAH&regionalschluessel=&sachmerkmal=WAM8&sachschluessel=WA03011900&sachmerkmal2=&sachschluessel2=&sachmerkmal3=&sachschluessel3=&stand=&auftrag=true&sprache=de"
response = requests.get(url)

In [20]:
json_data2 = xmltodict.parse(response.content)
table_data2 = json_data2['soapenv:Envelope']['soapenv:Body']['TabellenExportResponse']['TabellenExportReturn']['tabellen']['tabellen']['tabellenDaten']

In [21]:
df2 = pd.read_csv(StringIO(table_data2), sep=";", skiprows=5)

STEP 7: TIDY THE DATASET UP

In [22]:
#Rename Variables
df2.rename(columns={'Unnamed: 0': 'Year_Country', 'Unnamed: 1': 'Code', 'Unnamed: 2': 'Product', 'Ausfuhr: Besondere Maßeinheit': 'Export (number)', 'Ausfuhr: Gewicht': 'Export: Weight (t)', 'Ausfuhr: Wert': 'Export Value (Tsd. EUR)', 'Ausfuhr: Wert (US-Dollar)':'Export Value (Tsd. USD)','Einfuhr: Besondere Maßeinheit': 'Import (number)', 'Einfuhr: Gewicht': 'Import Weight (t)','Einfuhr: Wert': 'Import Value (Tsd. EUR)', 'Einfuhr: Wert (US-Dollar)':'Import Value (Tsd. USD)' },inplace=True)

In [23]:
#Drop second row (which has no values)
df2.drop(0, inplace=True)

In [24]:
#Drop last three rows (which have no values)
end_index = df2.query("Year_Country == '__________'").index[0]
df2 = df2.iloc[:end_index]
df2 = df2[df2["Year_Country"] != "__________"]

In [25]:
#Create a separate variable "year"
df2['Year'] = df2['Year_Country']
df2['Year'] = df2['Year'].str.replace(r"^(?!20).*$",'NaN',regex=True)
df2['Year'].replace(['NaN'], np.nan, inplace=True)
df2['Year'] = df2['Year'].ffill()

/var/folders/z0/7y9h036n67l9188jq204ym500000gn/T/ipykernel_2638/908341353.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Year'].replace(['NaN'], np.nan, inplace=True)


In [26]:
#Get rid of more rows (the ones with only years in it)
years = ['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
df2 = df2[~df2['Year_Country'].isin(years)]
df2.rename(columns={'Year_Country':'Country'},inplace=True)

In [27]:
#Replace Numbers in European format to US format
df2['Import Weight (t)'] = df2['Import Weight (t)'].replace(r'\,', '.', regex=True)
df2['Export: Weight (t)'] = df2['Export: Weight (t)'].replace(r'\,', '.', regex=True)

In [28]:
#Replace "-" with NaN (according to the Statistical Office, "-" means either NaN or exactly zero)
df2.replace(['-'], np.nan, inplace=True)

/var/folders/z0/7y9h036n67l9188jq204ym500000gn/T/ipykernel_2638/3540600199.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2.replace(['-'], np.nan, inplace=True)


In [29]:
#Keep only variables I want to analyze
df2 = df2[['Country','Year','Import Value (Tsd. USD)','Import Weight (t)']]

In [30]:
#Drop rows with NAs
df2.dropna(subset=['Import Value (Tsd. USD)', 'Import Weight (t)'], inplace=True)

In [31]:
#Cut off the "(ab xxx)"
df2['Country'] = df2['Country'].str.replace(r"\s*\(ab.*?\)",'',regex=True)

In [32]:
#Translate country names from German to English
unique_countries = df2['Country'].unique()
translation_dict = {country: GoogleTranslator(source='de', target='en').translate(country) for country in unique_countries}
df2['Country'] = df2['Country'].replace(translation_dict)
df2.head(25)
#Some corrections still to make manually
df2['Country'] = df2['Country'].str.replace(r'(Christmas island)','Chistmas Island',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(Egypt)','Egypt, Arab Rep.',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(Federated states of micronesia)','Micronesia, Fed. Sts.',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(French-Polynesia)','French Polynesia',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(Jibuti)','Djibouti',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(United States of America)','United States',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(Vietnam)','Viet Nam',regex=True)
df2['Country'] = df2['Country'].str.replace(r'(Czech Republic)','Czechia',regex=True)

In [33]:
#Filter from 2013 to 2023
years = ['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
df2 = df2[df2['Year'].isin(years)]

In [34]:
#Group by Country (Import Value)
df2['Import Value (Tsd. USD)'] = df2['Import Value (Tsd. USD)'].astype(float)
df_importvalue = df2.groupby('Country')['Import Value (Tsd. USD)'].sum().to_frame().sort_values(by="Import Value (Tsd. USD)",ascending=False)

In [35]:
#Group by Country (Import Weight)
df2['Import Weight (t)'] = df2['Import Weight (t)'].astype(float)
df_importweight = df2.groupby('Country')['Import Weight (t)'].sum().to_frame().sort_values(by="Import Weight (t)",ascending=False)

In [36]:
#Merge datasets df_importvalue and df_importweight
df_countries = df_importvalue.merge(df_importweight, left_on='Country', right_on='Country', how='outer')

STEP 8: SAVE THE DATASETS AS CSV

In [37]:
df_countries.to_csv('fishes_countries.csv', index=True)

STEP 9: ADD GDP TO TABLE

I'll use the data on GDP per Capita from the Worldbank, as of June 2025, https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)

In [38]:
#Read data
df_gdp = pd.read_csv('gdp_percapita_raw.csv',skiprows=3)

In [39]:
#Keep only the needed variables
df_gdp = df_gdp[['Country Name','2023']]

In [40]:
#Rename the variables
df_gdp.rename(columns={'Country Name': 'Country', '2023': 'GDP per Capita (2023)'},inplace=True)

STEP 10: MERGE GDP WITH OTHER DATAFRAMES

In [41]:
df_countrygdp = df_countries.merge(df_gdp, left_on='Country', right_on='Country', how='left')

STEP 11: SAVE MERGED DATASET TO CSV

In [42]:
df_countrygdp.to_csv('fishes_countrygdp.csv', index=True)